In [ ]:
def generate_combined_report_mass_organized(df, save_directory):
    """
    Generate combined PDF progress reports for Elite and Hybrid athletes, organized by Status, Trainer, Level, School/Org, and Grad Year.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        save_directory (str): Directory to save the PDFs.

    Returns:
        None
    """
    from matplotlib.backends.backend_pdf import PdfPages
    from datetime import datetime
    import os

    # Get the current date for folder and file naming
    current_date = datetime.now().strftime("%b%Y").upper()  # e.g., 'JAN2025'
    folder_date = datetime.now().strftime("%Y-%m-%d")  # e.g., '2025-01-02'

    # Filter for Elite and Hybrid members only
    valid_statuses = ['Elite', 'Hybrid']
    statuses = [status for status in df['Status'].unique() if status in valid_statuses]

    failed_reports = []  # List to collect athlete IDs for failed reports

    for status in statuses:
        # Filter athletes by Status
        status_df = df[df['Status'] == status]
        
        for athlete_id in status_df['ID'].unique():
            try:
                # Retrieve athlete's data
                athlete_data = status_df[status_df['ID'] == athlete_id]
                athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
                
                # Determine Trainer, Level, School/Org, Grad Year, and other attributes
                trainer = athlete_data['Trainer'].iloc[0] if 'Trainer' in athlete_data.columns else 'Unknown Trainer'
                level = athlete_data['Level'].iloc[0] if 'Level' in athlete_data.columns else 'Unknown'
                school_org = athlete_data['School/Org'].iloc[0] if 'School/Org' in athlete_data.columns else 'Unknown School'
                grad_year = athlete_data['Grad Year'].iloc[0] if 'Grad Year' in athlete_data.columns else 'Unknown Year'
                position = athlete_data['Position'].iloc[0] if 'Position' in athlete_data.columns else 'Unknown'
                dob = athlete_data['DOB'].iloc[0] if 'DOB' in athlete_data.columns else 'Unknown DOB'
                height = athlete_data['Height'].iloc[0] if 'Height' in athlete_data.columns else 'Unknown'
                weight = athlete_data['Weight'].iloc[0] if 'Weight' in athlete_data.columns else 'Unknown'
                created_date = datetime.now().strftime("%Y-%m-%d")

                # Find most recent phase for Strength workouts
                strength_data = athlete_data[athlete_data['Workout Type'] == 'Strength']
                if not strength_data.empty:
                    recent_strength_workout = strength_data.sort_values(by=['Test Number', 'Date'], ascending=[False, False]).iloc[0]
                    phase_name = recent_strength_workout['Phase'] if 'Phase' in recent_strength_workout else 'Unknown Phase'
                else:
                    phase_name = 'No Phase'
                
                # Build folder hierarchy
                if level in ['High School', 'College']:
                    folder_path = os.path.join(
                        save_directory, 
                        f"{status}_{current_date}",
                        trainer,
                        level,
                        school_org,
                        str(grad_year)
                    )
                else:  # Professionals or others without Level/Grad Year
                    folder_path = os.path.join(
                        save_directory,
                        f"{status}_{current_date}",
                        trainer,
                        "Professional" if level == "Professional" else "Other"
                    )
                
                # Create folders if they don't exist
                os.makedirs(folder_path, exist_ok=True)

                # Create the PDF filename
                pdf_filename = f"{athlete_name}, {phase_name}, {current_date}.pdf"
                pdf_path = os.path.join(folder_path, pdf_filename)
                
                # Generate the SCOPE report for the athlete
                with PdfPages(pdf_path) as pdf:
                    # Generate athlete details
                    details_text = (
                        f"Athlete: {athlete_name}\n"
                        f"Membership: {status}\n"
                        f"Trainer: {trainer}\n"
                        f"DOB: {dob}    H/W: {height}\"/{weight}lbs \n"
                        f"{school_org}  {grad_year}\n"
                        f"Position: {position}\n"
                        f"Created Date: {created_date}"
                    )

                    # Generate report sections
                    percentiles, athlete_name, athlete_birth_year = calculate_percentiles(df, athlete_id, metric_dict)

                    generate_cover_page(df, athlete_id, pdf, details_text)
                    radar_chart_percentiles(df, percentiles, athlete_name, athlete_id, pdf, unit_dict)

                    selected_metrics = [
        ('Weigh-in', 'Weigh-in', 'Weight'),
        ('Weigh-in', 'Height', 'Height'),
        ('Trackman Bullpen', 'Fastball', 'Max Velo'),
        ('Trackman Bullpen', 'Fastball', 'Average Velo'),
        ('Blast Motion', 'Front Toss', 'Max Bat Speed'),
        ('Blast Motion', 'Front Toss', 'Peak Hand Speed'),
        ('HitTrax', 'Front Toss', 'Average EV'),
        ('HitTrax', 'Front Toss', 'Max EV'),
        ('HitTrax', 'Front Toss', 'Max Distance'),
        ('Roll Ins', '5oz', 'Max Velo'),
        ('Double Plays', '5oz', 'Max Velo'),
        ('Turn and Burns', '5oz', 'Max Velo'),
        ('Pulldowns', '5oz', 'Max Velo'),
        ('Jump', 'Broad', 'Distance'),
        ('Jump', 'Vertical', 'Vert'),
        ('Back Squat', 'Strength-Speed', 'Weight'),
        ('Deadlift', 'Strength-Speed', 'Weight'),
        ('Bench Press', 'Strength-Speed', 'Weight'),
        ('ArmCare', 'Fresh Exam', 'Arm Score'),
                    ]

                    custom_labels = [
                        "Body Weight",
                        "Height",
                        "Fastball Max Velocity",
                        "Fastball Average Velocity",
                        "Max Bat Speed",
                        "Peak Hand Speed",
                        "Average Exit Velo",
                        "Max Exit Velo",
                        "Max Distance",
                        "5oz Roll Ins",
                        "5oz Double Plays",
                        "5oz Turn and Burns",
                        "5oz Pulldown",
                        "Broad Jump Distance",
                        "Vertical Jump Height",
                        "Speed Back Squat",
                        "Speed Deadlift",
                        "Speed Bench Press",
                        "ArmCare Score",
                    ]

                    generate_metric_table(df, athlete_id, selected_metrics, unit_dict, pdf, custom_labels)
                    bar_chart_improvement(df, athlete_id, metric_dict, unit_dict, pdf)
                    generate_strength_radar_chart(df, athlete_id, pdf)
                    generate_vbt_progress_graphs(df, athlete_id, pdf)
                    plot_jump_testing(df, athlete_id, pdf)
                    plot_balance_graph_with_labels(df, athlete_id, pdf)
                    plot_velocity_with_arm_score(df, athlete_id, pdf)

            except Exception:
                failed_reports.append(athlete_id)

    # Print summary of failures
    if failed_reports:
        print("\nFailed to generate reports for the following athlete IDs:")
        for athlete_id in failed_reports:
            print(f"- {athlete_id}")
    else:
        print("\nAll reports generated successfully.")


generate_combined_report_mass_organized(data, save_directory='C:/Users/benoi/OneDrive/Desktop/bea/Progress Summaries/')



